In [ ]:
# TODOS
# Add Rician objective function for LSQ fitting.
# rewrite for experiment

"""
(c) Stefano B. Blumberg and Paddy J. Slator, do not redistribute or modify

Code to replicate the ADC experiment (alongside matlab code - maybe translate to python?) <Add paper link>

Overview for cells:
    - Choose data size splits 2
    - Generate data examples 3-A/B/C
    - Data format for JOFSTO 4
    - Option to pass data directly, or save to disk and load 5-A/B
    - JOFSTO hyperparameters 6,7,8
"""

In [ ]:
########## (1)
# Import modules, see requirements.txt for jofsto requirements, set global seed

import numpy as np
import os
import sys
sys.path.append(os.path.abspath('..'))
import os, yaml 
from jofsto_code.jofsto_main import return_argparser, run

import matplotlib.pyplot as plt

np.random.seed(0)  # Random seed for entire script

In [ ]:
#Directories and filenames to save data 

#REPLACE WITH LOCATION OF JOFSTO INSTALLATION
basedir = '/Users/paddyslator/python/ED_MRI/'

#REPLACE WITH PATH TO SIMULATION GROUND TRUTH PARAMETERS
#array is n_voxels by n_model_parameters 
simulation_gt_parameters_path = basedir + 'output/adc_simulations/adc_simulations_n_train_1000_SNR_20_parameters_gt_full.npy'

#REPLACE WITH PATH TO SIMULATION GROUND TRUTH SIGNALS 
#THESE SHOULD BE WITH "SUPERDESIGN" ACQUISITION - HIGHLY OVERSAMPLING THE ACQUISITION PARAMETER SPACE
#array is n_voxels by n_acquisitions
simulation_gt_signals_path = basedir + 'output/adc_simulations/adc_simulations_n_train_1000_SNR_20_signals_super_full.npy'

#REPLACE WITH PATH TO SUPER-DESIGN ACQUISITION PARAMETERS
#array is n_acquisitions by n_acqusition_parameters, e.g. 288 (n_acquisitions) by 4 (gx, gy, gz, b) for HCP data
acq_params_super_signals_path = basedir + 'output/adc_simulations/adc_simulations_n_train_1000_SNR_20_acq_params_super.npy'


#load them
parameters = np.load(simulation_gt_parameters_path)
signals = np.load(simulation_gt_signals_path)
acq_params_super = np.load(acq_params_super_signals_path)



In [ ]:
########## (2)
# Data split sizes

n_samples = np.shape(signals)[0] #get the total number of simulated samples, reduce for faster training speed 

n_train = 8 * n_samples // 10 # No. training voxels
n_val = n_samples // 10  # No. validations set voxels
n_test = n_samples // 10  # No. test set voxels


#choose the size of the super-design
C_bar = 192


In [ ]:
########## (4)
# Load data into JOFSTO format

# Data in JOFSTO format, \bar{C} measurements, M target regresors
data = dict(
    train=signals[0:n_train,:],  # Shape n_train x \bar{C}
    train_tar=parameters[0:n_train,:],  # Shape n_train x M
    val=signals[n_train:(n_train + n_val),:],  # Shape n_val x \bar{C}
    val_tar=parameters[n_train:(n_train + n_val),:],  # Shape n_val x M
    test=signals[(n_train + n_val):(n_train + n_val + n_test),:],  # Shape n_test x \bar{C}
    test_tar=parameters[(n_train + n_val):(n_train + n_val + n_test),:],  # Shape n_test x M
)

#with open(os.path.dirname(__file__) + "/base.yaml", "r") as f:
#with open("/home/blumberg/Bureau/z_Automated_Measurement/Code/base.yaml", "r") as f:
with open(os.path.join(basedir, "base.yaml"), "r") as f:
    jofsto_args =  yaml.safe_load(f)

In [ ]:
########## (5-A)
# Option to save data to disk, and JOFSTO load
    
data_fil = os.path.splitext(simulation_gt_signals_path)[0] + '_jofsto_processed_data.npy'
#data_fil = "/home/blumberg/Bureau/z_Automated_Measurement/Output/paddy/adc_simulations.npy"
#data_fil = "/Users/paddyslator/python/ED_MRI/adc_simulations.npy"  # Add path to save file
np.save(data_fil, data)
print("Saving data as", data_fil)
pass_data = None

jofsto_args["--data_fil"] = data_fil


########## (5-B)
# Option to pass data to JOFSTO directly

pass_data = data

In [ ]:
########## (6)
# Simplest version of JOFSTO, modifying the most important hyperparameters


# Decreasing feature subsets sizes for JOFSTO to consider
jofsto_args["jofsto_train_eval"]["C_i_values"] = [C_bar, C_bar // 2, C_bar // 4, C_bar // 8, C_bar // 16]

# Feature subset sizess for JOFSTO evaluated on test data
jofsto_args["C_i_eval"] = [C_bar // 2, C_bar // 4, C_bar // 8, C_bar // 16]

# Scoring net C_bar -> num_units_score[0] -> num_units_score[1] ... -> C_bar units
jofsto_args["num_units_score"] = [1000, 1000]

# Task net C_bar -> num_units_task[0] -> num_units_task[1] ... -> M units
jofsto_args["num_units_task"] = [1000, 1000]

jofsto_args["out_base"] =  os.path.join(basedir, "test1")  #"/Users/paddyslator/python/ED_MRI/test1" #"/home/blumberg/Bureau/z_Automated_Measurement/Output/paddy"
jofsto_args["proj_name"] = "adc"
jofsto_args["run_name"] = "test"

jofsto_args["save_output"] = True

#jofsto_args["total_epochs"] = 1000


JOFSTO_output = run(args=jofsto_args, pass_data=pass_data)


In [ ]:
#load the FULL JOFSTO output
JOFSTO_output = np.load(os.path.join(basedir,jofsto_args["out_base"],jofsto_args["proj_name"],"results", jofsto_args["run_name"] + "_all.npy"), allow_pickle=True).item()




In [ ]:
#extract some useful parameters fom the jofsto output
#final subset index
C_last = JOFSTO_output["C_i_eval"][-1]
#index of the chosen acquisition parameters
acq_params_JOFSTO_index = JOFSTO_output[C_last]['measurements']
#chosen acquisition parameters
acq_params_JOFSTO = acq_params_super[acq_params_JOFSTO_index]

print('JOFSTO chosen acquisition parameters are: ' + str(acq_params_JOFSTO))


In [ ]:
#plot the signals at the super design and the JOFSTO chosen for a single voxel
#

#if the number of acquisition parameters is bigger than one, need to choose which one to plot on the x axis
if (acq_params_super.ndim > 1): 
    if (acq_params_super.shape[1] > 1):
        acq_param_to_plot = 1
        acq_params_super_to_plot = acq_params_super[:,acq_param_to_plot]
        acq_params_JOFSTO_to_plot = acq_params_JOFSTO[:,acq_param_to_plot]
else:
    acq_params_super_to_plot = acq_params_super
    acq_params_JOFSTO_to_plot = acq_params_JOFSTO
    
voxel_to_plot = 100


plt.plot(acq_params_super_to_plot, signals[voxel_to_plot,:], 'x')
plt.plot(acq_params_JOFSTO_to_plot, signals[voxel_to_plot,acq_params_JOFSTO_index], 'o')

plt.title('signals from voxel ' + str(100))
plt.legend(('Super design', 'JOFSTO chosen'))
plt.ylabel('signal')
plt.xlabel('acquisition parameter')


In [ ]:
#TO DO: make a function that can take new data as input and apply the JOFSTO NN to estimate the model parameters

In [ ]:
########## (7)
# Modify more JOFSTO hyperparameters, less important, may change results


In [ ]:

########## (8)
# Deep learning training hyperparameters for inner loop